In [5]:
import psycopg2
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from category_encoders import TargetEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import xgboost as xgb
from google.colab import userdata
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from pytorch_tabnet.tab_model import TabNetRegressor

# Загрузка данных
def load_data():
    db_host = userdata.get('db_host')
    db_port = userdata.get('db_port')
    db_name = userdata.get('db_name')
    db_user = userdata.get('db_user')
    db_password = userdata.get('db_password')

    if db_port and isinstance(db_port, str):
        db_port = int(db_port)

    connection = psycopg2.connect(
        host=db_host,
        port=db_port,
        database=db_name,
        user=db_user,
        password=db_password
    )

    tables = ["addresses", "developers", "offers", "offers_details",
              "realty_details", "realty_inside", "realty_outside"]

    dfs = {}
    for table in tables:
        query = f"SELECT * FROM public.{table};"
        dfs[table] = pd.read_sql(query, connection)

    connection.close()

    return dfs

# Функция очистки данных
def clean_data(dfs):
    dfs['addresses']['lat'] = dfs['addresses']['coordinates'].apply(lambda x: x['lat'] if isinstance(x, dict) else None)
    dfs['addresses']['lng'] = dfs['addresses']['coordinates'].apply(lambda x: x['lng'] if isinstance(x, dict) else None)
    dfs['addresses'].drop(columns=['id', 'coordinates', 'address', 'created_at', 'updated_at'], inplace=True)
    dfs['developers'].drop(columns=['id', 'created_at', 'updated_at'], inplace=True)
    dfs['offers'].drop(columns=['id', 'created_at', 'updated_at'], inplace=True)
    dfs['offers_details'].drop(columns=['id', 'created_at', 'updated_at'], inplace=True)
    dfs['realty_details']['finish_year'] = dfs['realty_details']['finish_date'].apply(lambda x: x.get('year') if isinstance(x, dict) else None)
    dfs['realty_details'].drop(columns=['id', 'finish_date', 'created_at', 'updated_at'], inplace=True)
    dfs['realty_inside'].drop(columns=['id', 'created_at', 'updated_at'], inplace=True)
    dfs['realty_outside'].drop(columns=['id', 'created_at', 'updated_at'], inplace=True)

    main_df = dfs['addresses'].merge(dfs['offers'], on='cian_id', how='inner') \
                              .merge(dfs['offers_details'], on='cian_id', how='inner')

    tables_to_left_join = ['developers', 'realty_details', 'realty_inside', 'realty_outside']
    for table in tables_to_left_join:
        main_df = main_df.merge(dfs[table], on='cian_id', how='left')

    return main_df

# Функция очистки выбросов
def remove_outliers_soft(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 3 * IQR
    upper_bound = Q3 + 3 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Функция обработки пропущенных значений
def handle_missing_values(df):
    df.fillna({
        'loggias': 0, 'separated_wc': 0, 'balconies': 0, 'combined_wc': 0, 'passenger_lifts': 0,
        'total_rate': df['total_rate'].mean(),
        'review_count': df['review_count'].mean(),
        'ceiling_height': df['ceiling_height'].mean(),
        'build_year': df['finish_year']
    }, inplace=True)

    mean_proportion_living_area = (df['living_area'] / df['total_area']).mean()
    df['living_area'].fillna(df['total_area'] * mean_proportion_living_area, inplace=True)

    mask = (df['total_area'] - df['living_area']) != 0
    mean_proportion_kitchen_area = (df.loc[mask, 'kitchen_area'] / (df.loc[mask, 'total_area'] - df.loc[mask, 'living_area'])).mean()
    df['kitchen_area'].fillna((df['total_area'] - df['living_area']) * mean_proportion_kitchen_area, inplace=True)

    df.drop(columns=['finish_year'], inplace=True)

    return df

# Функция кодирования категориальных переменных
def encode_features(df):
    onehot_columns = ['county', 'flat_type', 'sale_type', 'category', 'material_type', 'travel_type']
    target_columns = ['district', 'project_type', 'metro']
    ordinal_columns = {'repair_type': {'no': 0, 'cosmetic': 1, 'euro': 2, 'design': 3}}

    onehot_encoder = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')
    df_onehot = pd.DataFrame(onehot_encoder.fit_transform(df[onehot_columns]), columns=onehot_encoder.get_feature_names_out(onehot_columns))

    for col, mapping in ordinal_columns.items():
        df[col] = df[col].map(mapping)

    target_encoder = TargetEncoder()
    df_target = target_encoder.fit_transform(df[target_columns], df['price'])

    df.drop(columns=onehot_columns + target_columns, inplace=True)
    df = pd.concat([df, df_onehot, df_target], axis=1)

    return df

# Функция удаления нечисловых колонок
def remove_non_numeric(df):
    non_numeric_columns = df.select_dtypes(include=['object']).columns
    df = df.drop(columns=non_numeric_columns)
    return df

# Функция обработки данных
def process_data():
    dfs = load_data()
    df = clean_data(dfs)

    columns_to_clean = ['price', 'total_area', 'living_area', 'kitchen_area', 'rooms_count']
    for column in columns_to_clean:
        df = remove_outliers_soft(df, column)

    df = handle_missing_values(df)
    df = encode_features(df)
    df = remove_non_numeric(df)

    df = df.dropna(subset=['price'])

    X = df.drop(columns=['price'])
    y = df['price']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    X_train.fillna(X_train.mean(), inplace=True)
    X_test.fillna(X_test.mean(), inplace=True)

    return X_train, X_test, y_train, y_test

# Запуск обработки данных
X_train, X_test, y_train, y_test = process_data()


<ipython-input-5-49c99577ebd4>:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[table] = pd.read_sql(query, connection)
<ipython-input-5-49c99577ebd4>:91: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['living_area'].fillna(df['total_area'] * mean_proportion_living_area, inplace=True)
<ipython-input-5-49c99577ebd4>:95: FutureWarning: A value is trying to be set on a

In [6]:
X_train.shape

(92017, 62)

In [7]:
y_train = y_train.to_numpy().reshape(-1, 1)
y_test = y_test.to_numpy().reshape(-1,1)

In [8]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

In [9]:
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)


In [10]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [11]:
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32)


In [12]:
class MLPRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, x):
        return self.model(x)

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_dim = X_train.shape[1]
model = MLPRegressor(input_dim = input_dim, hidden_dim=64).to(device)

In [15]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(20):
    model.train()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")



Epoch 1, Loss: 0.0452
Epoch 2, Loss: 0.1572
Epoch 3, Loss: 0.1386
Epoch 4, Loss: 0.0640
Epoch 5, Loss: 0.0422
Epoch 6, Loss: 0.0387
Epoch 7, Loss: 0.0459
Epoch 8, Loss: 0.0289
Epoch 9, Loss: 0.0355
Epoch 10, Loss: 0.0774
Epoch 11, Loss: 0.0813
Epoch 12, Loss: 0.0572
Epoch 13, Loss: 0.0431
Epoch 14, Loss: 0.0714
Epoch 15, Loss: 0.0880
Epoch 16, Loss: 0.0219
Epoch 17, Loss: 0.2654
Epoch 18, Loss: 0.0857
Epoch 19, Loss: 0.0813
Epoch 20, Loss: 0.0412


In [16]:
model.eval()
preds = []
targets = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        output = model(X_batch)

        preds.append(output.cpu())
        targets.append(y_batch.cpu())

# Объединяем
preds = torch.cat(preds).numpy()
targets = torch.cat(targets).numpy()

# Возвращаем в исходный масштаб
preds_unscaled = scaler_y.inverse_transform(preds)
targets_unscaled = scaler_y.inverse_transform(targets)

# Метрики
mse = ((preds_unscaled - targets_unscaled) ** 2).mean()
mape = mean_absolute_percentage_error(targets_unscaled, preds_unscaled)
r2 = r2_score(targets_unscaled, preds_unscaled)

print(f"\nTest MSE: {mse:.4f}")
print(f"Test MAPE: {mape:.2%}")
print(f"Test R²: {r2:.4f}")


Test MSE: 23341461143552.0000
Test MAPE: 14.84%
Test R²: 0.9016


In [ ]:
table_model = TabNetRegressor(verbose=1, device_name= 'cuda')

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


In [ ]:
table_model.fit(
    X_train=X_train, y_train=y_train,
    eval_set=[(X_test, y_test)],
    eval_metric=['mse'],
    max_epochs=100,
    patience=10,
    batch_size=256,
    virtual_batch_size=128
)

# 4. Предсказание
preds = table_model.predict(X_test)
preds_inv = scaler_y.inverse_transform(preds)
y_test_inv = scaler_y.inverse_transform(y_test)

# 5. Метрики
mse = mean_squared_error(y_test_inv, preds_inv)
r2 = r2_score(y_test_inv, preds_inv)
mape = mean_absolute_percentage_error(y_test_inv, preds_inv)

print(f"Test MSE: {mse:.4f}")
print(f"Test R²: {r2:.4f}")
print(f"Test MAPE: {mape:.2%}")

epoch 0  | loss: 0.32781 | val_0_mse: 0.17941 |  0:00:11s
epoch 1  | loss: 0.20597 | val_0_mse: 0.16506 |  0:00:22s
epoch 2  | loss: 0.17822 | val_0_mse: 0.14735 |  0:00:33s
epoch 3  | loss: 0.17137 | val_0_mse: 0.14296 |  0:00:44s
epoch 4  | loss: 0.15932 | val_0_mse: 0.13937 |  0:00:54s
epoch 5  | loss: 0.15169 | val_0_mse: 0.14945 |  0:01:04s
epoch 6  | loss: 0.149   | val_0_mse: 0.12758 |  0:01:15s
epoch 7  | loss: 0.14253 | val_0_mse: 0.16372 |  0:01:26s
epoch 8  | loss: 0.13923 | val_0_mse: 0.12929 |  0:01:36s
epoch 9  | loss: 0.13612 | val_0_mse: 0.12528 |  0:01:46s
epoch 10 | loss: 0.13325 | val_0_mse: 0.13894 |  0:01:56s
epoch 11 | loss: 0.13185 | val_0_mse: 0.12209 |  0:02:07s
epoch 12 | loss: 0.1272  | val_0_mse: 0.16139 |  0:02:16s
epoch 13 | loss: 0.12893 | val_0_mse: 0.11659 |  0:02:26s
epoch 14 | loss: 0.12439 | val_0_mse: 0.11182 |  0:02:36s
epoch 15 | loss: 0.13093 | val_0_mse: 0.1595  |  0:02:46s
epoch 16 | loss: 0.12919 | val_0_mse: 0.15072 |  0:02:57s
epoch 17 | los

/usr/local/lib/python3.11/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Test MSE: 23397748162420.6562
Test R²: 0.9013
Test MAPE: 15.70%


In [18]:
from deepkan import DeepKAN
model_kan = DeepKAN(input_dim= X_train.shape[1], hidden_layers=[64, 64, 1]).to(device)

for epoch in range(20):
    model_kan.train()
    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 0.0326
Epoch 2, Loss: 0.0326
Epoch 3, Loss: 0.0520
Epoch 4, Loss: 0.0305
Epoch 5, Loss: 0.0321
Epoch 6, Loss: 0.0104
Epoch 7, Loss: 0.0294
Epoch 8, Loss: 0.0353
Epoch 9, Loss: 0.0525
Epoch 10, Loss: 0.1136
Epoch 11, Loss: 0.0748
Epoch 12, Loss: 0.0627
Epoch 13, Loss: 0.1025
Epoch 14, Loss: 0.0361
Epoch 15, Loss: 0.0559
Epoch 16, Loss: 0.0447
Epoch 17, Loss: 0.0325
Epoch 18, Loss: 0.0401
Epoch 19, Loss: 0.0896
Epoch 20, Loss: 0.0719


In [19]:
model_kan.eval()
preds = []
targets = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        output = model(X_batch)

        preds.append(output.cpu())
        targets.append(y_batch.cpu())

preds = torch.cat(preds).numpy()
targets = torch.cat(targets).numpy()

preds_unscaled = scaler_y.inverse_transform(preds)
targets_unscaled = scaler_y.inverse_transform(targets)

mse = ((preds_unscaled - targets_unscaled) ** 2).mean()
mape = mean_absolute_percentage_error(targets_unscaled, preds_unscaled)
r2 = r2_score(targets_unscaled, preds_unscaled)

print(f"\nTest MSE: {mse:.4f}")
print(f"Test MAPE: {mape:.2%}")
print(f"Test R²: {r2:.4f}")


Test MSE: 23317098528768.0000
Test MAPE: 14.58%
Test R²: 0.9017
